In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

from scipy.stats import uniform, randint, loguniform
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBRegressor

from fit_util import *

In [8]:
# X, y = load_data()
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=98987)

train_df, target = load_data()
X_train = train_df.iloc[np.random.choice(np.arange(len(train_df)), 300, False)]
y_train = target[X_train.index]

In [10]:
parameters = {
    "learning_rate": loguniform(5e-3,  1e-1),
    "max_depth": [10, 100, 200, 500, 1000, 3000, 4000, 5000],
    "n_estimators": [1, 10, 20, 100, 1000, 5000, 7000, 9000, 10000],
    "max_depth": [185],
    "n_estimators": [7374],
    "lambda" : loguniform(1e-4, 10),
    "min_child_weight" : loguniform(0.1, 55),
    "gamma" : loguniform(1e-4, 10)
    }

grid_model = RandomizedSearchCV(XGBRegressor(seed=0), parameters, cv=3, n_jobs=-1)
grid_model.fit(X_train, y_train);

In [11]:
print(grid_model.best_params_)

{'gamma': 0.0016045778808074773, 'lambda': 0.00036724291788078, 'learning_rate': 0.007004227601420743, 'max_depth': 185, 'min_child_weight': 19.357551391851082, 'n_estimators': 7374}


In [12]:
gs_model = XGBRegressor(**grid_model.best_params_, seed=0)

X, y = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=98987)

gs_model.fit(X_train, y_train);

In [14]:
evaluate(gs_model, X_train, y_train, "train ")
evaluate(gs_model, X_test, y_test, "test ")

train RMSE: 0.007959740429678418
test RMSE: 0.01121679405869426
